In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
###第二波改进增加多个命名空间



###载入数据集
mnist= input_data.read_data_sets('MNIST_data',one_hot=True)
###运行次数
max_steps =1001
###图片数量
image_num=30000
#文件路径
DIR='F:/PycharmDoc/'

###定义会话
sess=tf.Session()

###载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding'
                        



##来个函数
def variable_summariess(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)





###命名空间
with tf.name_scope('input'):
    ###定义连个placeholder
    x= tf.placeholder(tf.float32,[None,784],name='x_input')
    y= tf.placeholder(tf.float32,[None,10],name='y_input')

###显示图片
###其中-1代表着不确定的值，1为维度因为是黑白的图片
with tf.name_scope('input_reshape')
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)
                        
                        
                        
###创建一个简单的神经网络

with tf.name_scope('layer'):
    with tf.name_scope('Weight'):
        W= tf.Variable(tf.zeros([784,10]),name='Weight_1')
        variable_summariess(W)
    with tf.name_scope('biases'):
        b=tf.Variable(tf.zeros([10]))
        variable_summariess(b)
    with tf.name_scope('mut'):
        mut=tf.matmul(x,W)+b
    with tf.name_scope('prediction'):
        prediction= tf.nn.softmax(mut)


###二次代价函数
with tf.name_scope('loss'):
    
# loss= tf.reduce_mean(tf.square(y- prediction))
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
###使用梯度下降法
with tf.name_scope('train'):
    train= tf.train.GradientDescentOptimizer(0.2).minimize(loss)

###初始化变量
init= tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    ###结果存在在一个布尔型的变量中
    ###tf.argmax是用来判断其中的最大值
    with tf.name_scope('correct_prediction'):
        correct_prediciton = tf.equal(tf.arg_max(y,1),tf.arg_max(prediction,1))
    ###求准确率
    ###tf.cast是用来转化的
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediciton,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
                        
###产生metadata文件
if tf.gfile.Exists(DIR+'projector/projector/meta.tsv'):
    tf.gfile.DeleteRecursively（DIR+'projector/projector/meta.tsv'）
with open(DIR+'projector/projector/meta.tsv','w') as f:
    labels = sess.run(tf.arg_max(mnist.test.labels[:],1))
    for i in range(image_num):
        f.writer(str(label[i]+'\n'))
                        
                        
####合并所有的summary
merged=tf.summary.merge_all()
                        
projector_writer =tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver = tf.train.Saver()
config=projector.ProjectorConfig()
embed= config.embeddings.add()
embed.tensor_name=embedding.name
embed.metadata_path= DIR+'projector/projector/meta.tsv'
embed.sprite.image_path= DIR +'projector/data/mnist_10K_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)
                        
                        
sess.run(init)

for i in range(max_steps):
###首先获得一个批次，大小为100
    batch_xs,batch_ys= mnist.train.next_batch(100)
    run_options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata=tf.RunMetadata()
    summary,_=sess.run([merged,train],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary,epoch)    
        
    if i%100 ==0:
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter'+str(i)+'Testing Accuracy'+str(acc))
                        
                        
                        
saver.save(sess,DIR+'projector/projector/a_model.ckpt',global_step=max_steps))
projector_writer.close()
sess.close()